In [9]:
# Needed libraries
#
import numpy as np
from numba import jit, njit, prange

#numba.config.DEBUG_ARRAY_OPT=1
#numba.config.NUMBA_PARALLEL_DIAGNOSTICS=1

from scipy.io import FortranFile
from pylab import *
from matplotlib import pyplot as plt
from scipy import stats
import time
import sys
import os
import importlib
import pickle
import h5py

import cle202_utils_phil as cle202_utils   # there is a cle.py that must be in the same
                    # directory as this file
    
np.set_printoptions(precision=14,suppress=False)
plt.rcParams["figure.figsize"] = [6.5,5]
matplotlib.rcParams.update({'font.size': 9})

# colorbar function to have nice colorbars in figures
def colorbar(mappable):
   from mpl_toolkits.axes_grid1 import make_axes_locatable
   import matplotlib.pyplot as plt
   last_axes = plt.gca()
   ax = mappable.axes
   fig = ax.figure
   divider = make_axes_locatable(ax)
   cax = divider.append_axes("right", size="3.5%", pad=0.05)
   cbar = fig.colorbar(mappable, cax=cax)
   #cbar.formatter.set_powerlimits((4, 9))
   plt.sca(last_axes)
   return cbar
%matplotlib widget

In [10]:
#with open('obsstokes_avg.pkl','rb') as f:
#    f1aa,f2aa,sobs_a,yobs_a,rms,wvl,xxl,yyl = pickle.load(f)
# with open('obsstokes_avg_new.pkl','rb') as f:
#     f1aa,f2aa,sobs_a,yobs_a,rms,wvl,xxl,yyl = pickle.load(f)    
    

In [11]:
##3dipole pickle cube
with open('obsstokes_3dipole.pkl','rb') as f:
    sobs_a,yobs_a,rms,xxl,yyl = pickle.load(f)  

In [12]:
# plt.figure()
# asa=plt.imshow(sobs_a[:,:,4])
# colorbar(asa)
# #plt.imshow(yobs_a)
# print(rms,wvl,xxl,yyl)
# print(sobs_a.shape)

In [14]:
importlib.reload(cle202_utils)

##Global constants:
verbose= np.int_(1)  ## not fully implememted
r2d    = 180./np.pi 

##main controling parameters
reduce = np.int_(1)  ## really fast only with reduced.
chi2   = np.int_(10) ## the maximum chi^2 to return; ideally a  normalized and reduced chi^2 should be <1
ns     = np.int_(4) ##maximum number of solutions to return
dbdir  = '/home/alin/Documents/physics_prog/cle/test_cle_degeneracy/db202_deg_150_fe13/'

out=np.zeros((xxl,yyl,ns,10),dtype=np.float64)     
iquv_match=np.empty((xxl,yyl,ns,8),dtype=np.float64)
#an array to store the maximum solutions from the DB inversion. 
#The inversion can produce less than ns solution due to the chi2 condition.
##outparams_a[3:5] are the b fields with phi and theta angles and outparams_a[6:8] are bx by bz scaled using the V_obs/V_db ratio
##outparams_a[9:10] are the DB index and chi2 values respectively.


## assume that the database config is unique at this point.
##read the header and dimensions and just pass them as parameters to function calls
g=cle202_utils.readhdr_par(dbdir+'db.hdr')
hdrparams=[cle202_utils.parsehdr_par(g)][0] ## dont understand why the 0 indexing is needed...
#print(hdrparams[1]*hdrparams[2]*hdrparams[3]*hdrparams[4]*hdrparams[12]*4)
#cgrid,ned,ngx,nbphi,nbtheta,xed,gxmin,gxmax,bphimin,bphimax,bthetamin,bthetamax,nline,wavel=hdrparams

#start0=time.time()
database2=cle202_utils.sdbread_par(cle202_utils.find_elongation_par2(dbdir,yobs_a[0,0]),hdrparams) ##initialize the database variable

nam,num=cle202_utils.file_preprocess(dbdir)
print('DB read start')

fenc,database=cle202_utils.find_sel_db(yobs_a,
                                       hdrparams,
                                       nam,
                                       num,
                                       xxl,
                                       yyl)   

#dt= "{:4.3f}".format(time.time()-start0)
#print(dt,' SECONDS FOR DB READ ',dt)
print('DB read finalized')


#print(database[333356,:,:,:],database2[333356,:,:])
start0=time.time()
@jit(parallel=True)#,forceobj=True)#, nogil=True)    #
def run_twoline_par(sobs_a,database,fe,yobs_a,dbdir,rms,hdrparams,xxl,yyl,nsearch,reduced):
    if sobs_a.shape[-1] != 8: ## ARP: assuming the last dimension of sobs is the number of observables; 2 lines=8
        print('clematch: Must have a two line observation; Aborting!')
        return np.empty((xxl,yyl,nsearch,10),dtype=np.float64)
        
    outparams=np.empty((xxl,yyl,nsearch,10),dtype=np.float64)
    iquv_match=np.empty((xxl,yyl,nsearch,8),dtype=np.float64)
    
    index=np.empty((xxl,yyl,nsearch),dtype=np.int_)
    smatch=np.empty((xxl,yyl,nsearch,8),dtype=np.float64)
    chisq=np.empty((xxl,yyl,nsearch),dtype=np.float64)
    physics=np.empty((xxl,yyl,nsearch,8),dtype=np.float64)

    print("start loop")
    for xx in range(xxl):
        print("ext loop:",xx)
        #start0=time.time()
        for yy in prange(yyl):
            #start0=time.time()
            #print(database[:,:,:,filenc[xx,yy]].shape)
            #tmp=np.ascontiguousarray(database[:,:,:,0])
            #print(tmp.shape,db.shape, np.array_equal(tmp,db))
            index[xx,yy,:],smatch[xx,yy,:,:],chisq[xx,yy,:],physics[xx,yy,:,:]=cle202_utils.clematch_par(sobs_a[xx,yy,:],
                                                                                                         yobs_a[xx,yy],
                                                                                                         database[fe[xx,yy]],
                                                                                                         hdrparams,
                                                                                                         rms,
                                                                                                         '',
                                                                                                         chi2,
                                                                                                         nsearch,
                                                                                                         reduced,
                                                                                                         verbose)
            #index,sdb,chisq,physics=cle202_utils.clematch_par(sobs_a[xx,yy,:],yobs_a[xx,yy],database2,hdrparams,rms,'',10,4,reduced,verbose)
            #dt= "{:4.3f}".format(time.time()-start0)
            #print(dt,' SECONDS FOR large ',dt)
            #database2=database[:,:,:,filenc[xx,yy]]
            #start0=time.time()
#             index[xx,yy,:],smatch[xx,yy,:,:],chisq[xx,yy,:],physics[xx,yy,:,:]=cle202_utils.clematch_par(sobs_a[xx,yy,:],
#                                                                                                          yobs_a[xx,yy],
#                                                                                                          db,
#                                                                                                          hdrparams,
#                                                                                                          rms,
#                                                                                                          '',
#                                                                                                          chi2,
#                                                                                                          nsearch,
#                                                                                                          reduced,
#                                                                                                          verbose)
            #dt= "{:4.3f}".format(time.time()-start0)
            #print(dt,' SECONDS FOR small ',dt)
            #for zz in prange(0,nsearch):
            #    outparams[xx,yy,0:6,zz]=cle202_utils.physa_par(index[zz],yobs_a[xx,yy],hdrparams,1)
            outparams[xx,yy,:,0]=index[xx,yy,:]
            outparams[xx,yy,:,1]=chisq[xx,yy,:]
            outparams[xx,yy,:,2:10]=physics[xx,yy,:,:]
           # outparams[xx,yy,:,:]=cle202_utils.clematch_par(sobs_a[xx,yy,:],yobs_a[xx,yy],database2,hdrparams,rms,'',chi2,nsearch,reduced,verbose)

            
#             if (len(index.shape) == 0):
#                 outparams[xx,yy,:,:]=np.empty((11,nsearch),dtype=np.float64)
#             elif (index.shape[0] >= 4):
#                 for zz in prange(0,4):
#                     #ARP: need to merge phys and physa to get a unified/simplified output in physics variable.
#                     outparams[xx,yy,0:6,zz]=cle202_utils.physa_par(index[zz],yobs_a[xx,yy],hdrparams,1)
#                     #outparams[xx,yy,6:9,zz]=(cle202_utils.phys_par(index[zz],yobs_a[xx,yy],hdrparams,sobs_a[xx,yy,3]/sdb[zz][3]))[3:6]
#                     outparams[xx,yy,6:9,zz]=physics[xx,yy,3:6,zz]
#                     #print(physics)
#                     #print((cle202_utils.phys_par(index[zz],yobs_a[xx,yy],hdrparams,sobs_a[xx,yy,3]/sdb[zz][3]))[3:6])
#                     outparams[xx,yy,9,zz]=chisq[xx,yy,zz]
#                     outparams[xx,yy,10,zz]=index[xx,yy,zz]
#             else:
#                 for zz in prange(0,index.shape[0]):
#                     outparams[xx,yy,0:6,zz]=cle202_utils.physa_par(index[zz],yobs_a[xx,yy],hdrparams,1)
#                     #outparams[xx,yy,6:9,zz]=(cle202_utils.phys_par(index[zz],yobs_a[xx,yy],hdrparams,sobs_a[xx,yy,3]/sdb[zz][3]))[3:6]
#                     outparams[xx,yy,6:9,zz]=physics[xx,yy,3:6,zz]
#                     outparams[xx,yy,9,zz]=chisq[xx,yy,zz]
#                     outparams[xx,yy,10,zz]=index[xx,yy,zz]

        #dt= "{:4.3f}".format(time.time()-start0)
        #print(dt,' SECONDS FOR ',xxl,'profile set: ',dt)
    return outparams,smatch

out,iquv_match=run_twoline_par(sobs_a,database,fenc,yobs_a,dbdir,rms,hdrparams,xxl,yyl,ns,reduce)

dt= "{:4.3f}".format(time.time()-start0)
print(dt,' SECONDS FOR DBINVERT ',dt)

##ARP: original working version of the parralel code, keep for testing and compatibility.
#@njit(parallel=True)#,forceobj=True, nogil=True)    #
# def run_twoline_par(sobs_a,yobs_a,dbdir,rms,g,hdrparams,xxl,yyl):
#     #start0=time.time()
#     if sobs_a.shape[-1] != 8: ## ARP: assuming the last dimension of sobs is the number of observables; 2 lines=8
#         print('clematch: Must have a two line observation; Aborting!')
#         return np.empty((xxl,yyl,11,4),dtype=np.float64)
        
#     outparams=np.empty((xxl,yyl,11,4),dtype=np.float64)
#     fildb_prev='bla'
#     #database=cle202_utils.sdbread_par(cle202_utils.find_elongation_par(dbdir,yobs_a[0,0]),dbdir,yobs_a[0,0]) ##initialize the database variable

#     for xx in prange(0,xxl):
#         print("bla:",xx)
#         for yy in prange(0,yyl):
#             fildb=cle202_utils.find_elongation_par(dbdir,yobs_a[xx,yy])
#             if not fildb == fildb_prev:
#                 database=cle202_utils.sdbread_par(fildb,dbdir,hdrparams,yobs_a[xx,yy])
#             #print(sobs_a[xx,yy,:],database.shape[0]*database.shape[1]*database.shape[2])  
            
#             #     if numlines != nline:
# #         print("Must be the same number of lines in observations and database")
# #         print('numlines=',numlines,'nline=',nline)
# #         #sys.exit()

#             index,sdb,chisq,physics=cle202_utils.clematch_par(sobs_a[xx,yy,:],yobs_a[xx,yy],database,hdrparams,rms,dbdir,'',10,4,0,1)
#             #print(index.shape,len(index.shape),sdb,chisq,physics)
#             if (len(index.shape) == 0):
#                 fildb_prev=fildb
#             elif (index.shape[0] >= 4):
#                 for zz in prange(0,4):
#                     outparams[xx,yy,0:6,zz]=cle202_utils.physa_par(index[zz],yobs_a[xx,yy],hdrparams,1)
#                     outparams[xx,yy,6:9,zz]=(cle202_utils.phys_par(index[zz],yobs_a[xx,yy],hdrparams,sobs_a[xx,yy,3]/sdb[zz][3]))[3:6]
#                     outparams[xx,yy,9,zz]=chisq[zz]
#                     outparams[xx,yy,10,zz]=index[zz]
#             else:
#                 for zz in prange(0,index.shape[0]):
#                     outparams[xx,yy,0:6,zz]=cle202_utils.physa_par(index[zz],yobs_a[xx,yy],hdrparams,1)
#                     outparams[xx,yy,6:9,zz]=(cle202_utils.phys_par(index[zz],yobs_a[xx,yy],hdrparams,sobs_a[xx,yy,3]/sdb[zz][3]))[3:6]
#                     outparams[xx,yy,9,zz]=chisq[zz]
#                     outparams[xx,yy,10,zz]=index[zz]
#             fildb_prev=fildb      ##update the previous database. 

#     #dt= "{:4.3f}".format(time.time()-start0)
#     #print(dt,' SECONDS FOR TOTAL ',dt)
#     return outparams




#run_twoline_par(sobs_a,yobs_a,dbdir,rms,hdrparams,xxl,yyl,ns)
#run_twoline_par.parallel_diagnostics(level=4)
#print (out.shape)
#plt.imshow(out[:,:,8,0])
#
# end of example


DB read start
0.390306  SECONDS FOR elong num  0.390306
read actual db
13.876  SECONDS FOR dbread  13.876
0.140  SECONDS FOR encoding  0.140
DB read finalized


<ipython-input-14-672abe373ce3>:52: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "run_twoline_par" failed type inference due to: Failed in nopython mode pipeline (step: nopython frontend)
NameError: name 'r2d' is not defined
During: resolving callee type: type(CPUDispatcher(<function clematch_par at 0x7f786bd3bef0>))
During: typing of call at <ipython-input-14-672abe373ce3> (83)

During: resolving callee type: type(CPUDispatcher(<function clematch_par at 0x7f786bd3bef0>))
During: typing of call at <ipython-input-14-672abe373ce3> (83)


File "<ipython-input-14-672abe373ce3>", line 83:
def run_twoline_par(sobs_a,database,fe,yobs_a,dbdir,rms,hdrparams,xxl,yyl,nsearch,reduced):
    <source elided>
                                                                                                         nsearch,
                                                                                                         reduced)#,
            

start loop


/home/alin/anaconda3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "run_twoline_par" was compiled in object mode without forceobj=True.

File "<ipython-input-14-672abe373ce3>", line 67:
def run_twoline_par(sobs_a,database,fe,yobs_a,dbdir,rms,hdrparams,xxl,yyl,nsearch,reduced):
    <source elided>
    print("start loop")
    for xx in range(xxl):
    ^

  state.func_ir.loc))
/home/alin/anaconda3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-14-672abe373ce3>", line 67:
def run_twoline_par(sobs_a,database,fe,yobs_a,dbdir,rms,hdrparams,xxl,yyl,nsearch,reduced):
    <source elided>
    print("start

ext loop: 0


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mNameError: name 'r2d' is not defined[0m

In [14]:
print(out.shape)
plt.figure()
# # #plt.imshow(yobs_a.T)
im=plt.imshow(out[:,:,0,3].T,vmin=-10,vmax=10)
colorbar(im)
# plt.figure()
# im2=plt.imshow(yobs_a)
# colorbar(im2)
# # #print(ff)
# # plt.show()

(1024, 1024, 4, 10)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
##dump 3dipole
with open('outparams_3dipole_20210511.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
   pickle.dump(out, f)

In [6]:
##dump muram
# with open('outparams_avglos_par_20210511.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#    pickle.dump(out, f)

In [56]:
interp='none'
fig, plots = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
aa=plots[0,0].imshow(out[:,:,0,7],interpolation=interp,cmap='seismic',vmin=-100,vmax=100)
plots[0,0].set_title("DBINVERT Bx")
fig.colorbar(aa,ax=plots[0,0],label='B')

#outparams_a[300:700,:,6,0]=-outparams_a[300:700,:,6,0]
ab=plots[1,0].imshow(out[:,:,0,8]*2,interpolation=interp,cmap='seismic',vmin=-100,vmax=100)
plots[1,0].set_title("DBINVERT By")
fig.colorbar(ab,ax=plots[1,0],label='B')

ac=plots[2,0].imshow(out[:,:,0,9]*3,interpolation=interp,cmap='seismic',vmin=-100,vmax=100)
plots[2,0].set_title("DBINVERT Bz")
fig.colorbar(ac,ax=plots[2,0],label='B')


aa=plots[0,1].imshow(out[:,:,0,1],interpolation=interp,cmap='seismic')
plots[0,1].set_title("DBINVERT chi^2")
fig.colorbar(aa,ax=plots[0,1],label='chi^2')

#outparams_a[300:700,:,6,0]=-outparams_a[300:700,:,6,0]
ab=plots[1,1].imshow(out[:,:,0,2],interpolation=interp,cmap='seismic',vmin=7,vmax=9.5)
plots[1,1].set_title("DBINVERT Density")
fig.colorbar(ab,ax=plots[1,1],label='alog(ne)')

ac=plots[2,1].imshow(out[:,:,0,4],interpolation=interp,cmap='seismic')
plots[2,1].set_title("DBINVERT LOS pos")
fig.colorbar(ac,ax=plots[2,1],label='R_sun')
# ac=plots[2,1].imshow(out[:,:,0,2],interpolation=interp,cmap='seismic')
# plots[2,1].set_title("DBINVERT LOS Position")
# fig.colorbar(ac,ax=plots[2,1],label='R_sun')


ac=plots[0,2].imshow(out[:,:,0,5],interpolation=interp,cmap='seismic')
plots[0,2].set_title("DBINVERT b_theta")
fig.colorbar(ac,ax=plots[0,2],label='theta')

ac=plots[1,2].imshow(out[:,:,0,6],interpolation=interp,cmap='seismic')
plots[1,2].set_title("DBINVERT B_phi")
fig.colorbar(ac,ax=plots[1,2],label='phi')

ac=plots[2,2].imshow(out[:,:,0,3],interpolation=interp,cmap='seismic')
plots[2,2].set_title("DBINVERT Height fit")
fig.colorbar(ac,ax=plots[2,2],label='R_sun')
plt.tight_layout()
plt.savefig('/home/alin/Desktop/CLE2.png',dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
im=plt.imshow(sobs_a[:,:,6])
colorbar(im)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#print(index[0:2])
# verbose=0
# try: verbose
# except:  
#     verbose=0
#     print("caca")
#print(fildb)
#charar = np.array((4,3,2),dtype=object)

#charar[1,2] = fildb
#charar[0,0] = fildb
#print(charar.shape)

print(np.argpartition([5,3,1,9,14,7,6,0],0))

print(np.argpartition([5,3,1,9,14,7,6,0],1))

print(np.argpartition([5,3,1,9,14,7,6,0],2))

print(np.argpartition([5,3,1,9,14,7,6,0],3))

print(np.argpartition([5,3,1,9,14,7,6,0],4))
print(np.argpartition([5,3,1,9,14,7,6,0],range(5)))
print(np.argsort([5,3,1,9,14,7,6,0]))

print(np.sort([5,3,1,9,14,7,6,0]))

astr=np.empty(4,dtype=np.int16)
bla=np.partition([5,3,1,9,14,7,6,0],range(4))
for i in range(4):
    #print(np.where([5,3,1,9,14,7,6,0] == bla[i] )[0][0])
    astr[i]=np.where([5,3,1,9,14,7,6,0] == bla[i] )[0][0]
    
print(astr)
#print(np.where([5,3,1,9,14,7,6,0] == np.partition([5,3,1,9,14,7,6,0],range(4))[0] ))
arra=np.indices([24])
print(arra)
shape1=[2,3,4]#,5,6,4]
print(shape1)
shape2=np.array((2,3,4))#,4,5,6,4])
print(shape2.shape)
shape3=np.reshape(arra,shape2)
print(shape3,shape3.shape)

[7 1 2 3 4 5 6 0]
[7 2 1 3 4 5 6 0]
[7 2 1 3 4 5 6 0]
[1 7 2 0 4 5 6 3]
[1 7 2 0 6 5 3 4]
[7 2 1 0 6 5 4 3]
[7 2 1 0 6 5 3 4]
[ 0  1  3  5  6  7  9 14]
[7 2 1 0]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]]
[2, 3, 4]
(3,)
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]] (2, 3, 4)
